In [1]:
import pandas as pd
import numpy as np
import nltk 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import string as st
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vivian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vivian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Vivian\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
df = pd.read_csv("./data/features_demands.csv")

# Text Lemmentalization

In [7]:
stopword = nltk.corpus.stopwords.words('english')
def pre_processing(text):
    a = [w for w in text if w not in st.punctuation]
    text = ''.join(a)
    texts = nltk.word_tokenize(text)
    a = [w for w in texts if w not in stopword]
    a = [x.lower() for x in a]
    a = [i for i in a if not i.isdigit()]
    lemmatizer = WordNetLemmatizer()
    a = [lemmatizer.lemmatize(w) for w in a]
    return ' '.join(a)

In [8]:
df = df[df['CleanText'].notna()]
df['CleanText'] = df['CleanText'].apply(pre_processing)

0        flying entourage w jeff parish surveying downe...
1                     air operation underway grand isle la
2        i signed volunteer helping man cooling station...
3        new york ag suggests emergency housing basemen...
4                                             dammit bobby
                               ...                        
42535    forecast td nine become saturday explicitly sh...
42536    it first official advisory likely opinion beco...
42537    btw upgraded will probably within next couple ...
42538                                          sorry sorry
42539    tomorrow year anniversary faced another storm ...
Name: CleanText, Length: 42540, dtype: object

# Label Based on Sentiment and Demand

In [11]:
sent = pd.read_csv("./data/sentiment140_sentiment.csv", index_col=0)

In [18]:
target_df = pd.merge(df, sent, on = "Tweet Id")[["Tweet Id", "TimeDiff", "tweet_1st", "tweet_2nd", "Demand", "Sentiment140_Sentiment"]]

In [19]:
target_df

,Tweet Id,TimeDiff,tweet_1st,tweet_2nd,Demand,Sentiment140_Sentiment
0,1433942885716615173,215.999722,10.708929,3.772147,Other,0
1,1433942716996485123,215.988611,11.942713,3.885051,Other,4
2,1433942626944884738,215.982778,6.072673,0.735367,Other,4
3,1433942451937501191,215.971111,8.767741,3.170668,Food,0
4,1433942167152697346,215.952222,2.240252,3.462578,Other,0
...,...,...,...,...,...,...
42535,1430909520868433920,15.108056,9.969593,4.999687,Other,0
42536,1430906539494154244,14.910556,8.256058,4.304694,Shelter,4
42537,1430904806038642696,14.795833,9.488568,2.686853,Other,4
42538,1430884811292487682,13.471667,2.373932,3.909397,Other,0


In [25]:
def label (row):
   if row['Demand']=="Water" and row['Sentiment140_Sentiment']!=0: # 4 positive
      return 0
   elif row['Demand']=="Water" and row['Sentiment140_Sentiment']==0:
      return 1
   elif row['Demand']=="Food" and row['Sentiment140_Sentiment']!=0: # 4 positive, 0 nagative
      return 0
   elif row['Demand']=="Food" and row['Sentiment140_Sentiment']==0:
      return 1
   elif row['Demand']=="Shelter" and row['Sentiment140_Sentiment']!=0: # 4 positive, 0 nagative
      return 0
   elif row['Demand']=="Shelter" and row['Sentiment140_Sentiment']==0:
      return 1
   elif row['Demand']=="Electricity" and row['Sentiment140_Sentiment']!=0: # 4 positive, 0 nagative
      return 0
   elif row['Demand']=="Electricity" and row['Sentiment140_Sentiment']==0:
      return 1
   elif row['Demand']=="Medicine" and row['Sentiment140_Sentiment']!=0: # 4 positive, 0 nagative
      return 0
   elif row['Demand']=="Medicine" and row['Sentiment140_Sentiment']==0:
      return 1

In [34]:
Water = target_df[target_df["Demand"] == 'Water']
Food = target_df[target_df["Demand"] == 'Food']
Electricity = target_df[target_df["Demand"] == 'Electricity']
Shelter = target_df[target_df["Demand"] == 'Shelter']
Medicine = target_df[target_df["Demand"] == 'Medicine']
needs = target_df[target_df["Demand"] != 'Other']

In [27]:
Water['Label'] = Water.apply (lambda row: label(row), axis=1)
Water = Water[["TimeDiff", "tweet_1st", "tweet_2nd", "Label"]]
Water.to_csv("./data/Water.csv")

<ipython-input-27-427aca4fed6e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Water['Label'] = Water.apply (lambda row: label(row), axis=1)


In [28]:
Food['Label'] = Food.apply (lambda row: label(row), axis=1)
Food = Food[["TimeDiff", "tweet_1st", "tweet_2nd", "Label"]]
Food.to_csv("./data/Food.csv")

<ipython-input-28-357250f74ec9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Food['Label'] = Food.apply (lambda row: label(row), axis=1)


In [29]:
Shelter['Label'] = Shelter.apply (lambda row: label(row), axis=1)
Shelter = Shelter[["TimeDiff", "tweet_1st", "tweet_2nd", "Label"]]
Shelter.to_csv("./data/Shelter.csv")

<ipython-input-29-b382d6997203>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Shelter['Label'] = Shelter.apply (lambda row: label(row), axis=1)


In [30]:
Electricity['Label'] = Electricity.apply (lambda row: label(row), axis=1)
Electricity = Electricity[["TimeDiff", "tweet_1st", "tweet_2nd", "Label"]]
Electricity.to_csv("./data/Electricity.csv")

<ipython-input-30-0f1001ca3cfc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Electricity['Label'] = Electricity.apply (lambda row: label(row), axis=1)


In [31]:
Medicine['Label'] = Medicine.apply (lambda row: label(row), axis=1)
Medicine = Medicine[["TimeDiff", "tweet_1st", "tweet_2nd", "Label"]]
Medicine.to_csv("./data/Medicine.csv")

<ipython-input-31-b33975809b35>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Medicine['Label'] = Medicine.apply (lambda row: label(row), axis=1)


In [33]:
Other = target_df[target_df["Demand"] == 'Other'][["TimeDiff", "tweet_1st", "tweet_2nd"]]
Other.to_csv("./data/Other.csv")

In [39]:
def label_need (row):
    if row['Sentiment140_Sentiment']==0:
        if row['Demand'] == "Water":
            return "Water"
        elif row['Demand'] == "Food":
            return "Food"
        elif row['Demand'] == "Shelter":
            return "Shelter"
        elif row['Demand'] == "Medicine":
            return "Medicine"
        elif row['Demand'] == "Electricity":
            return "Electricity"
    else:
        return "No Need"

needs = target_df[target_df["Demand"] != 'Other']
needs['Label'] = needs.apply (lambda row: label_need(row), axis=1)
needs = needs[["TimeDiff", "tweet_1st", "tweet_2nd", "Label"]]
needs.to_csv("./data/needs.csv")

<ipython-input-39-a1c8ced9effa>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  needs['Label'] = needs.apply (lambda row: label_need(row), axis=1)
